In [1]:
!pip install -U ipywidgets
!pip install -U google-genai
!pip install -q transformers accelerate peft langdetect tqdm pandas huggingface_hub
!pip install -U optuna
!pip install -U bitsandbytes
!pip install -U flash_attn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 96.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.4/719.4 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 27.6 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.43.0
    Uninstalling google-auth-2.43.0:
      Successfully uninsta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 50.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 141.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=253780426 sha256=4e2f9e39313266b1544b68138b15b91ee6221eccf14f7902b7c6620351340810
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash_attn


In [ ]:
from huggingface_hub import HfFolder
HfFolder.save_token("")

import warnings
warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    module="torch.utils.checkpoint"
)

In [3]:
# ─────────────────────────────── IMPORTS ───────────────────────────────
import re, json
import logging
from pathlib import Path
from typing import Sequence, List
from concurrent.futures import ProcessPoolExecutor

from tqdm.auto import tqdm
import pandas as pd
try:
    from langdetect import detect, LangDetectException
    HAS_LANGDETECT = True
except ImportError:
    HAS_LANGDETECT = False
    logging.warning("langdetect not installed. Language filtering disabled.")

from google import genai
from google.genai import types

In [ ]:
#!/usr/bin/env python3
# Gaza-stance classification (Gemini) – v6, prompt “StanceClassifier-Gaza-v2”
# --------------------------------------------------------------------------
# • Reconstruit le texte complet depuis les CSV Facebook & Instagram.
# • Filtre les posts non anglophones puis échantillonne 0,5 %.
# • Envoie chaque post à Gemini-2.5-flash-preview avec le prompt ci-dessous.
# • La réponse doit être **exactement** un label sur une ligne.
# • Sauvegarde CSV : text, gpt_category
# --------------------------------------------------------------------------

# ─────────────────────────────── 0. LOGGING ────────────────────────────────
"""import logging, os, sys
logging.basicConfig(level=logging.WARNING,           # root silencieux
                    format="[%(levelname)s] %(message)s",
                    force=True)                      # ← reset complet

log = logging.getLogger("pipeline")                  # ton logger
log.setLevel(logging.INFO)                           # un seul handler (root)
# pas d’ajout de handler, on laisse propager au root

for name in ("google", "google_genai", "google.api_core"):
    logging.getLogger(name).setLevel(logging.ERROR)  # stop INFO/DEBUG

# ─────────────────────────────── 1. CONFIG ────────────────────────────────
FACEBOOK_CSV  = "/home/lisst_ai/DATA/FACEBOOK_concat.csv"
#INSTAGRAM_CSV = "/home/lisst_ai/DATA/INSTAGRAM_concat.csv"

#GOOGLE_API_KEY  = ""
#MODEL_NAME      = "gemini-2.5-flash-preview-04-17"
OUTPUT_FILENAME = "/home/lisst_ai/DATA/gaza_stance_sampled_classified.csv"

SAMPLE_FRAC      = 0.005   # 0,5 %
RANDOM_STATE     = 42
TARGET_LANG      = "en"
NUM_WORDS_SAMPLE = 100
NUM_WORKERS      = None

SEP             = " - "
CONSTRUCTED_COL = "_text"
ALLOWED_CATEGORIES = {
    "Pro-Palestinian",
    "Pro-Israeli",
    "Anti-War_Pro-Peace",
    "Other",
    "Off-topic",
}

"""

# ─────────────────────────────── 3. PROMPT ────────────────────────────────
SYSTEM_INSTRUCTION = """
You are “StanceClassifier-Gaza-v2”.
Task: read one English-language social-media post about the 2023-2025 Gaza War and
output **exactly one** label from the list below.

Think step-by-step **silently** (don’t reveal your reasoning).
Return only the label on a single line, nothing else.

### Allowed labels
1. Pro-Palestinian
2. Pro-Israeli
3. Anti-War_Pro-Peace
4. Other
5. Off-topic          ← use when the post is not about the Gaza War at all

### Definitions & guidance

**Pro-Palestinian**
Any stance that primarily supports Palestinians or blames Israel.
✔ Detect even subtle cues: 🇵🇸 emoji, “from the river to the sea”, focus on Palestinian victims, words like “genocide”, “occupation”, “apartheid”, or praise for resistance.
✘ A single humanitarian mention of both sides → see Anti-War_Pro-Peace unless Israel is clearly blamed.

**Pro-Israeli**
Primarily supports Israel or blames Hamas/Palestinian side.
✔ Cues: 🇮🇱, “right to self-defence”, “terrorist organisation”, hostage hashtags, emphasis on Hamas using human shields.
✘ Balanced calls for restraint → Anti-War_Pro-Peace unless Hamas is clearly condemned more than Israel.

**Anti-War_Pro-Peace**
Core message = stop the violence / protect civilians on *both* sides, with balanced language.

**Other** (use sparingly)
Mentions the war but no discernible leaning **after** checking for:
‣ sarcasm or irony
‣ vocabulary that allocates guilt (even implicitly)
‣ spotlight on one population’s suffering
‣ selective historical references

**Off-topic**
Post lacks substantive reference to the Gaza War (generic memes, ads, ambiguous “Pray for them” with no context, etc.).

### Checklist before choosing “Other”
1. Does the wording, emoji, or hashtag tilt sympathy toward one side?
2. Are perpetrators or victims named asymmetrically?
3. Is there an implied moral judgment?
If **any** answer is yes, assign the corresponding stance; else, Other.

### Tie-break rules
* Stance + peace call → keep the stance.
* Balanced criticism but one side more severe → assign that side.
* Unsure between Anti-War and partisan → choose the partisan stance.

### Output format
Return exactly one of:
Pro-Palestinian | Pro-Israeli | Anti-War_Pro-Peace | Other | Off-topic

""".strip()
"""
# ─────────────────────────────── 4. GEMINI ────────────────────────────────
log.info("Initialisation API Gemini…")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
client = genai.Client(api_key=GOOGLE_API_KEY)

GEN_CFG = types.GenerateContentConfig(
    system_instruction=SYSTEM_INSTRUCTION,
    temperature=0.1,
    safety_settings=[
        types.SafetySetting(category=c, threshold=types.HarmBlockThreshold.BLOCK_NONE)
        for c in (
            types.HarmCategory.HARM_CATEGORY_HARASSMENT,
            types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
            types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
            types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        )
    ],
    thinking_config=genai.types.ThinkingConfig(
      thinking_budget=0
    )
)
log.info("Modèle prêt : %s", MODEL_NAME)

# ─────────────────────────────── 5. UTILITAIRES ────────────────────────────
ZERO_WIDTH_RE = re.compile(r"[\u200B-\u200F\u202A-\u202E\u2060-\u206F\uFEFF]")

def strip_invisible(t: str) -> str:
    return ZERO_WIDTH_RE.sub("", t)

def concatenate_fields(values: Sequence[str | float | None],
                       *, sep: str = SEP) -> str:
    parts: List[str] = []
    for v in values:
        if isinstance(v, str):
            v = v.strip()
            if v and v.lower() not in sep.join(parts).lower():
                parts.append(v)
    return sep.join(parts)

def safe_read_csv(path: str | Path) -> pd.DataFrame:
    log.info("Lecture CSV : %s", path)
    try:
        df = pd.read_csv(path, low_memory=False)
    except pd.errors.ParserError:
        df = pd.read_csv(path, engine="python", on_bad_lines="skip")
    log.info(" → %s lignes", f"{len(df):,}")
    return df

def process_facebook(df: pd.DataFrame) -> pd.DataFrame:
    cols = ["Message", "Description", "Image Text", "Link Text"]
    df[CONSTRUCTED_COL] = df.apply(lambda r: concatenate_fields(
        [r.get(c) for c in cols]), axis=1)
    return df

def process_instagram(df: pd.DataFrame) -> pd.DataFrame:
    cols = ["Description", "Image Text"]
    df[CONSTRUCTED_COL] = df.apply(lambda r: concatenate_fields(
        [r.get(c) for c in cols]), axis=1)
    return df

# ───────────── détection langue (multiprocessing) ──────────────
def _detect(sample: str) -> str | None:
    try:
        return detect(sample)
    except Exception:
        return None

def _lang_worker(args):
    i, txt = args
    sample = " ".join(txt.split()[:NUM_WORDS_SAMPLE])
    return i, (_detect(sample) == TARGET_LANG)

def filter_english(texts: list[str]) -> list[bool]:
    workers = NUM_WORKERS or (os.cpu_count() or 4)
    flags = [False] * len(texts)
    with ProcessPoolExecutor(max_workers=workers) as ex:
        for i, ok in ex.map(_lang_worker, enumerate(texts), chunksize=512):
            flags[i] = ok
    return flags

# ────────────────────────────── 6. CLASSIFY ───────────────────────────────
def classify(post: str) -> str:
    try:
        resp = client.models.generate_content(
            model=MODEL_NAME,
            contents=post,
            config=GEN_CFG,
        )
    except Exception as e:
        log.error("Gemini error : %s – exit.", e)
        sys.exit(1)

    if not resp.text:
        return "INVALID"

    label = resp.text.splitlines()[0].strip()
    return label if label in ALLOWED_CATEGORIES else "INVALID"

# ───────────────────────────── 7. PIPELINE ────────────────────────────────
def main() -> None:
    df_fb = process_facebook(safe_read_csv(FACEBOOK_CSV))
    #df_ig = process_instagram(safe_read_csv(INSTAGRAM_CSV))

    log.info("Filtre de langue : %d %d", len(df_fb), len(df_ig))  # avant le filtrage
    for df in (df_fb, df_ig):
        mask = filter_english(df[CONSTRUCTED_COL].tolist())
        df.drop(index=df.index[~pd.Series(mask)], inplace=True)

    log.info("Après filtre de langue : %d %d", len(df_fb), len(df_ig))

    df_sampled = pd.concat([
        df_fb.sample(frac=SAMPLE_FRAC, random_state=RANDOM_STATE),
        df_ig.sample(frac=SAMPLE_FRAC, random_state=RANDOM_STATE)
    ], ignore_index=True)
    log.info("Échantillon : %s posts", f"{len(df_sampled):,}")

    texts = df_sampled[CONSTRUCTED_COL].apply(strip_invisible).tolist()
    if not texts:
        raise RuntimeError("Aucun texte anglais après filtrage.")

    #log.info("Classification Gemini…")
    #cats = [classify(t) for t in tqdm(texts, desc="Gemini", unit="post")]

   #out = pd.DataFrame({"text": texts, "gpt_category": cats})
    #n_inv = (out["gpt_category"] == "INVALID").sum()
    #if n_inv:
    #    log.warning("%d réponses INVALID", n_inv)

    #out.query("gpt_category != 'INVALID'").to_csv(OUTPUT_FILENAME, index=False)
    log.info("Fichier enregistré : %s", OUTPUT_FILENAME)

# ──────────────────────────── Entrée script ───────────────────────────────
if __name__ == "__main__":
    log.info("Lancement pipeline…")
    main()
    log.info("Terminé.")
    """

In [4]:
EFF_BATCH_TARGET = 32

# ───────────────────────────────────────────────────────────────
# Llama-3-70B-Instruct • LoRA • 4-bit NF4 • Optuna • Cosine LR • MCC
# ───────────────────────────────────────────────────────────────
#   • Entrée tronquée à 512 tokens
#   • Quantisation bitsandbytes 4-bit NF4
#   • Fusion des classes : Other + Off-topic + Anti-War_Pro-Peace → Neutral
# ───────────────────────────────────────────────────────────────

import os, gc, random, threading, _thread, json, math, copy
from pathlib import Path
from typing import List

import numpy as np
import torch, torch.nn as nn
from torch.utils.data import Dataset

from google.colab import drive
drive.mount('/content/drive')

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, matthews_corrcoef

import optuna
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    EvalPrediction,
    set_seed,
)
from peft import LoraConfig, get_peft_model
import pandas as pd

# ╭───────────────────────────────╮
SEED = 42
set_seed(SEED); random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

USE_GRADIENT_CHECKPOINTING = True
MODEL_NAME = "mistralai/Ministral-8B-Instruct-2410"
NUM_EPOCHS, EARLY_STOP_PATIENCE, TIMEOUT_TRAIN = 40, 6, 3 * 36000
KFOLD, N_TRIALS = 5, 3
artifact_dir = Path("./outputs"); artifact_dir.mkdir(exist_ok=True, parents=True)
SINGLE_FOLD_ONLY = False
MAX_LENGTH = 512
# ╰───────────────────────────────╯

# ───────────── Lecture du CSV + fusion -------------------------
CSV_PATH = "/content/drive/MyDrive/UTJ2/Memoire/Scripts/Stance/gaza_stance_sampled_classified.csv"
df = pd.read_csv(CSV_PATH)          # colonnes: text, gpt_category / cat

if "gpt_category" in df.columns:
    df.rename(columns={"gpt_category": "cat"}, inplace=True)

# ─── Fusion des classes mineures → Neutral
FUSE_MAP = {
    "Pro-Palestinian":        "Pro-Palestinian",
    "Pro-Israeli":            "Pro-Israeli",
    "Other":                  "Neutral",
    "Off-topic":              "Neutral",
    "Anti-War_Pro-Peace":     "Neutral",
}
df["cat"] = df["cat"].map(FUSE_MAP)             # applique la fusion
df.dropna(subset=["cat"], inplace=True)         # sécurité : lignes non mappées

# ───────────── Mappings label ↔ id ------------------------------
cats      = sorted(df["cat"].unique())          # ['Neutral', 'Pro-Israeli', 'Pro-Palestinian']
label2id  = {c: i for i, c in enumerate(cats)}
id2label  = {i: c for c, i in label2id.items()}
df["label_id"] = df["cat"].map(label2id)

# ───────────── Diagnostics --------------------------------------
print(f"{len(df):,} exemples chargés (classes fusionnées).")
print("Répartition des classes :", df["label_id"].value_counts().to_dict())
print("label2id :", label2id)

# ───────────── Listes utiles / constantes -----------------------
messages   = df["text"].tolist()
labels     = df["label_id"].tolist()
NUM_LABELS = len(cats)                           # 3

# ───────────── Split train / val / test -------------------------
msg_tv, msg_te, lab_tv, lab_te = train_test_split(
    messages, labels, test_size=0.10, stratify=labels, random_state=SEED
)

Mounted at /content/drive
6,365 exemples chargés (classes fusionnées).
Répartition des classes : {2: 4021, 0: 1265, 1: 1079}
label2id : {'Neutral': 0, 'Pro-Israeli': 1, 'Pro-Palestinian': 2}


In [5]:
def enc(txts: List[str]):
    prompts = [build_prompt(t) for t in txts]
    return tok(                     # ← important
        prompts,
        truncation=True,
        max_length=MAX_LENGTH,
        add_special_tokens=False    # plus de double BOS
    )

def build_prompt(txt: str) -> str:
    """Prompt Mistral officiel, sûr pour l'encodeur HF."""
    messages = [
        {"role": "system",
         "content": f"You are an expert assistant. Classify the following text into one of these "
                    f"categories: {cats_string}. Respond with the category label only."},
        {"role": "user", "content": txt}
    ]
    return tok.apply_chat_template(
        messages,
        tokenize=False,                # on renvoie une string
        add_generation_prompt=False     # ajoute automatiquement le tag assistant
    ).strip()

# ───────────── Prompt instructif -----------------------------------------
cats_string = ", ".join(cats)

# ───────────── Tokenizer --------------------------------------------------
tok = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side="left")  # ← left‑pad plus sûr
if tok.pad_token_id is None or tok.pad_token_id == tok.eos_token_id:
    # 1) on crée un vrai token pad
    tok.add_special_tokens({'pad_token': '<pad>'})
    # 2) on pointe officiellement dessus
    tok.pad_token = '<pad>'

# IMPORTANT : redimensionner l’embedding AVANT d’injecter LoRA
VOCAB = len(tok)

class TokenizedDS(Dataset):
    def __init__(self, e, y):
        self.e, self.y = e, y
    def __len__(self):
        return len(self.y)
    def __getitem__(self, i):
        d = {k: torch.tensor(v[i]) for k, v in self.e.items()}
        d["labels"] = torch.tensor(self.y[i], dtype=torch.long)
        return d

# ───────────── WeightedCETrainer ----------------------------------------
class WeightedCETrainer(Trainer):
    def __init__(self, *args, class_weights: torch.Tensor, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights.float()

    # ▿ ajouter le nouvel argument (ou **kwargs) ▿
    def compute_loss(
        self,
        model,
        inputs,
        return_outputs: bool = False,
        num_items_in_batch: int | None = None,   # ← NEW
        **kwargs,                                # ← compatibilité future
    ):
        labels  = inputs["labels"]
        outputs = model(**{k: v for k, v in inputs.items() if k != "labels"})
        logits  = outputs.logits

        weight = self.class_weights.to(logits.device)
        loss = nn.functional.cross_entropy(
            logits,
            labels,
            weight=weight,    # pondération des classes
            reduction="mean"  # moyenne directe sur le batch
        )

        return (loss, outputs) if return_outputs else loss

collator = DataCollatorWithPadding(tok)

# ───────────── Metrics ----------------------------------------------------

def cat_metrics(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "f1_micro": f1_score(y_true, y_pred, average="micro"),
        "mcc": matthews_corrcoef(y_true, y_pred),
    }

def compute_metrics(ev: EvalPrediction):
    y_hat = np.argmax(ev.predictions, axis=1)
    g = cat_metrics(ev.label_ids, y_hat)
    return {
        "eval_accuracy": g["accuracy"],
        "eval_f1_macro": g["f1_macro"],
        "eval_f1_micro": g["f1_micro"],
        "eval_mcc": g["mcc"],
    }


te_enc = enc(msg_te)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [6]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [7]:
# ───────────── Timeout helper --------------------------------------------

def timeout_train(trainer, seconds):
    t = threading.Thread(target=trainer.train)
    t.start(); t.join(seconds)
    if t.is_alive():
        _thread.interrupt_main(); t.join()

In [ ]:
# ────────────────────────── Pré-requis ──────────────────────────
import os, gc, copy, json, math, random
import numpy as np
import torch
from sklearn.model_selection import StratifiedKFold, train_test_split
import optuna

# (on suppose que tous les imports HF, LoRA, votre collator, compute_metrics,
#  WeightedCETrainer, enc(), TokenizedDS, timeout_train(), etc. sont déjà présents)

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)

artifact_dir = "./artifacts"; os.makedirs(artifact_dir, exist_ok=True)

# ─────────────── Chargement et découpage du corpus ───────────────
df = pd.read_csv(CSV_PATH)
if "gpt_category" in df.columns:
    df = df.rename(columns={"gpt_category": "cat"})

cats       = sorted(df["cat"].unique())
label2id   = {c: i for i, c in enumerate(cats)}
id2label   = {i: c for c, i in label2id.items()}
df["label_id"] = df["cat"].map(label2id)

messages, labels = df["text"].tolist(), df["label_id"].tolist()
NUM_LABELS       = len(cats)

msg_tv, msg_te, lab_tv, lab_te = train_test_split(
    messages, labels, test_size=0.10, stratify=labels, random_state=SEED
)

# ─────────────────────── Fonctions utilitaires ───────────────────────
def build_model(r_lora, lora_dropout, use_bf16):
    """Construit un modèle QLoRA + classification, sans entraînement."""
    bnb_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16 if use_bf16 else torch.float16,
    )
    base = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=NUM_LABELS,
        device_map="auto",
        quantization_config=bnb_cfg,
        attn_implementation="flash_attention_2",
    )
    base.resize_token_embeddings(len(tok))
    base.config.pad_token_id      = tok.pad_token_id
    base.config.use_cache         = False
    base.config.use_paged_attention = True
    if USE_GRADIENT_CHECKPOINTING:
        base.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})

    l_cfg = LoraConfig(
        r=r_lora,
        lora_alpha=2 * r_lora,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="SEQ_CLS",
        target_modules=[
            "q_proj", "k_proj", "v_proj",
            "o_proj", "gate_proj", "up_proj", "down_proj",
        ],
    )
    model = get_peft_model(base, l_cfg)
    for p in model.parameters():
        if p.requires_grad: p.data = p.data.float()
    return model

# ──────────────── Optuna objective  (validation croisée) ────────────────
def objective(trial):
    # Hyper-paramètres
    lr            = trial.suggest_float("lr", 5e-5, 5e-4, log=True)
    bsz           = trial.suggest_categorical("bsz", [16, 32])
    r_lora        = trial.suggest_int("lora_r", 8, 32, step=8)
    lora_dropout  = trial.suggest_float("lora_dropout", 0.01, 0.1, log=True)

    grad_acc = max(1, math.ceil(EFF_BATCH_TARGET / bsz))
    use_bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    use_fp16 = not use_bf16
    skf      = StratifiedKFold(n_splits=KFOLD, shuffle=True, random_state=SEED)

    fold_mcc, best_fold_state, best_fold_metric = [], None, -1.0
    VAL_FOLDS_TO_USE = [0, 2]

    for fold_idx, (tr_idx, va_idx) in enumerate(skf.split(msg_tv, lab_tv)):
        if fold_idx not in VAL_FOLDS_TO_USE:
            continue

        # jeux train / val encodés
        msg_tr = [msg_tv[i] for i in tr_idx]; lab_tr = [lab_tv[i] for i in tr_idx]
        msg_va = [msg_tv[i] for i in va_idx]; lab_va = [lab_tv[i] for i in va_idx]
        train_ds = TokenizedDS(enc(msg_tr), lab_tr)
        val_ds   = TokenizedDS(enc(msg_va), lab_va)

        # pondération des classes
        counts = np.bincount(lab_tr, minlength=NUM_LABELS)
        cls_weights = torch.tensor(counts.sum() / (NUM_LABELS * (counts + 1e-9)), dtype=torch.float)

        model = build_model(r_lora, lora_dropout, use_bf16)

        args = TrainingArguments(
            output_dir=f"{artifact_dir}/trial_{trial.number}/fold_{fold_idx}",
            num_train_epochs=NUM_EPOCHS,
            per_device_train_batch_size=bsz,
            per_device_eval_batch_size=32,
            gradient_accumulation_steps=grad_acc,
            warmup_ratio=0.08,
            learning_rate=lr,
            lr_scheduler_type="cosine",
            bf16=use_bf16,
            fp16=use_fp16,
            gradient_checkpointing=USE_GRADIENT_CHECKPOINTING,
            eval_strategy="epoch",
            save_strategy="no",               # pas besoin de sauvegarde intermédiaire
            seed=SEED,
            logging_strategy="epoch",
            label_names=["labels"],
            load_best_model_at_end=False,
            metric_for_best_model="eval_mcc",
            greater_is_better=True,
        )

        trainer = WeightedCETrainer(
            model=model,
            args=args,
            train_dataset=train_ds,
            eval_dataset=val_ds,
            data_collator=collator,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=EARLY_STOP_PATIENCE)],
            class_weights=cls_weights,
        )

        # entraînement (protégé par timeout)
        timeout_train(trainer, TIMEOUT_TRAIN)
        mcc = trainer.evaluate(val_ds)["eval_mcc"]
        fold_mcc.append(mcc)

        if mcc >= best_fold_metric:
            best_fold_metric = mcc
            best_fold_state  = copy.deepcopy(model.state_dict())

        del trainer, model
        gc.collect(); torch.cuda.empty_cache()

    mean_mcc = float(np.mean(fold_mcc)) if fold_mcc else -1.0
    trial.set_user_attr("best_state", best_fold_state)
    return mean_mcc

# ─────────────── Lancement de la recherche Optuna ───────────────
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=N_TRIALS)

best_trial   = study.best_trial
best_params  = best_trial.params
best_state   = best_trial.user_attrs["best_state"]

# ─────────────── Reconstruction & chargement du meilleur modèle ───────────────
use_bf16     = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
best_model   = build_model(
    r_lora       = best_params["lora_r"],
    lora_dropout = best_params["lora_dropout"],
    use_bf16     = use_bf16,
)
best_model.load_state_dict(best_state)

# ─────────────────────── Préparation du jeu test ───────────────────────
test_ds = TokenizedDS(enc(msg_te), lab_te)
trainer = WeightedCETrainer(
    model=best_model,
    args=TrainingArguments(
        output_dir=f"{artifact_dir}/best_model_mcc",
        per_device_eval_batch_size=32,
        dataloader_drop_last=False,
        seed=SEED,
    ),
    data_collator=collator,
    compute_metrics=compute_metrics,
)

test_metrics = trainer.evaluate(test_ds)
print("Test MCC :", round(test_metrics["eval_mcc"], 4))
print("Test F1  :", round(test_metrics["eval_f1_macro"], 4))

# ────────────────────────── Sauvegarde finale ──────────────────────────
save_dir = f"{artifact_dir}/best_model_mcc"
best_model.save_pretrained(save_dir)
tok.save_pretrained(save_dir)
with open(os.path.join(save_dir, "best_params.json"), "w") as f:
    json.dump(best_params, f, indent=2)

print(f"Modèle et tokenizer sauvegardés dans « {save_dir} ».")

[I 2026-01-21 23:01:21,312] A new study created in memory with name: no-name-2abf9fd9-6779-49a3-b9ca-b158158b4d00


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of MinistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Ministral-8B-Instruct-2410 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,Mcc
1,2.693500,1.868276,0.460733,0.352298,0.460733,0.304958
2,1.557400,1.260357,0.675393,0.542386,0.675393,0.493513
3,1.174200,1.276580,0.572426,0.403626,0.572426,0.424154
4,0.994000,1.268189,0.677138,0.559448,0.677138,0.540091
5,0.993200,1.058420,0.678883,0.587438,0.678883,0.551001
6,0.813500,1.046070,0.730366,0.603411,0.730366,0.587101
7,0.805100,1.550747,0.735602,0.525211,0.735602,0.567049
8,0.770300,1.436279,0.794066,0.591447,0.794066,0.634763
9,0.626700,1.030996,0.746073,0.613571,0.746073,0.603847
10,0.641500,1.178468,0.658813,0.527824,0.658813,0.519469


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of MinistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Ministral-8B-Instruct-2410 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,Mcc
1,3.271700,2.014221,0.439791,0.260960,0.439791,0.231848
2,1.352300,1.431186,0.639616,0.435999,0.639616,0.438214
3,1.122700,1.296345,0.605585,0.468024,0.605585,0.458435
4,0.935900,1.170893,0.675393,0.525782,0.675393,0.522474
5,0.959500,1.446252,0.772251,0.561069,0.772251,0.576455
6,0.790300,1.217962,0.752182,0.591888,0.752182,0.598605
7,0.821900,1.161209,0.680628,0.537156,0.680628,0.534403
8,0.750400,1.314155,0.648342,0.512879,0.648342,0.488066
9,0.776700,1.569360,0.529668,0.503802,0.529668,0.428062
10,0.686000,1.265613,0.760908,0.595778,0.760908,0.596196


In [ ]:
best_params  = json.load(open(os.path.join(artifact_dir, "best_params.json")))
print(best_params)

In [ ]:
  Parameters: lr=3.15e-04, bsz=32, lora_r=24

In [ ]:
{'lr': 0.000315, 'bsz': 32, 'lora_r': 16, 'lora_dropout': 0.0}


In [ ]:
# ╭──────────────────────── Phase finale  ───────────────────────╮
# 1)   Re-chargement du CSV  (état vierge)
df = pd.read_csv(CSV_PATH)
if "gpt_category" in df.columns:
    df.rename(columns={"gpt_category": "cat"}, inplace=True)

cats      = sorted(df["cat"].unique())
label2id  = {c: i for i, c in enumerate(cats)}
id2label  = {i: c for c, i in label2id.items()}
df["label_id"] = df["cat"].map(label2id)
messages, labels = df["text"].tolist(), df["label_id"].tolist()
NUM_LABELS = len(cats)

# 2)   Découpage train / val / test
msg_tv, msg_te, lab_tv, lab_te = train_test_split(
    messages, labels, test_size=0.10, stratify=labels, random_state=SEED
)
msg_tr, msg_va, lab_tr, lab_va = train_test_split(
    msg_tv, lab_tv, test_size=0.10, stratify=lab_tv, random_state=SEED
)

# 3)   Encodage
tr_enc, va_enc, te_enc = enc(msg_tr), enc(msg_va), enc(msg_te)
train_ds = TokenizedDS(tr_enc, lab_tr)
val_ds   = TokenizedDS(va_enc, lab_va)
test_ds  = TokenizedDS(te_enc, lab_te)

# 4)   Hyper-paramètres Optuna retenus
best_params = json.load(open(artifact_dir / "best_params.json"))
best_params = {'lr': 0.000315, 'bsz': 32, 'lora_r': 64, 'lora_dropout': 0.1}

lr            = best_params["lr"]
bsz           = best_params["bsz"]
r_lora        = best_params["lora_r"]
lora_dropout  = best_params["lora_dropout"]

grad_acc = max(1, math.ceil(EFF_BATCH_TARGET / bsz))
use_bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
use_fp16 = not use_bf16

# 5)   Instanciation modèle + LoRA
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if use_bf16 else torch.float16,
)
base = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    device_map="auto",
    quantization_config=bnb_cfg,
    attn_implementation="flash_attention_2",
)
base.resize_token_embeddings(len(tok))
base.config.pad_token_id = tok.pad_token_id
base.config.use_cache = False
base.config.use_paged_attention = True
if USE_GRADIENT_CHECKPOINTING:
    base.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})

l_cfg = LoraConfig(
    r=r_lora,
    lora_alpha=2 * r_lora,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=[
        "q_proj", "k_proj", "v_proj",
        "o_proj", "gate_proj", "up_proj", "down_proj",
    ],
)
model = get_peft_model(base, l_cfg)
for p in model.parameters():
    if p.requires_grad:
        p.data = p.data.float()

# 6)   Pondération des classes
counts       = np.bincount(lab_tr, minlength=NUM_LABELS)
class_weights = torch.tensor(counts.sum() / (NUM_LABELS * (counts + 1e-9)), dtype=torch.float)

# 7)   Arguments d’entraînement finaux
args = TrainingArguments(
    output_dir=f"{artifact_dir}/final_run",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=bsz,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=grad_acc,
    warmup_ratio=0.08,
    learning_rate=lr,
    lr_scheduler_type="cosine",
    bf16=use_bf16,
    fp16=use_fp16,
    gradient_checkpointing=USE_GRADIENT_CHECKPOINTING,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_mcc",
    greater_is_better=True,
    seed=SEED,
    logging_strategy="epoch",
    label_names=["labels"],
)

trainer = WeightedCETrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=DataCollatorWithPadding(tok),
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=EARLY_STOP_PATIENCE)],
    class_weights=class_weights,
)

# 8)   Entraînement + évaluation finale
timeout_train(trainer, TIMEOUT_TRAIN)          # <-- garde la même limite que plus haut
print("Best val MCC:", trainer.state.best_metric)

test_metrics = trainer.evaluate(test_ds)
print("Test set :", {k: round(v, 4) for k, v in test_metrics.items()})

# (facultatif) Sauvegarde du modèle affiné
trainer.save_model(f"{artifact_dir}/final_model")
tok.save_pretrained(f"{artifact_dir}/final_model")
# ╰─────────────────────────────────────────────────────────────╯

In [ ]:
Test set : {'eval_accuracy': 0.7943, 'eval_f1_macro': 0.6463, 'eval_f1_micro': 0.7943, 'eval_mcc': 0.6533, 'eval_loss': 1.4161, 'eval_runtime': 55.0543, 'eval_samples_per_second': 11.57, 'eval_steps_per_second': 0.363, 'epoch': 16.0}


In [ ]:
# ───────────── Scores détaillés par classe ─────────────────────
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd

# 1) Prédictions brutes sur le test set
pred_res = trainer.predict(test_ds)
y_true   = pred_res.label_ids
y_pred   = np.argmax(pred_res.predictions, axis=1)

# 2) Rapport complet (precision / recall / f1 / support)
report = classification_report(
    y_true,
    y_pred,
    target_names=[id2label[i] for i in range(NUM_LABELS)],
    digits=4,
    output_dict=True          # ← pour l’avoir aussi sous forme dict / DataFrame
)
print("\n=== Classification report ===")
print(pd.DataFrame(report).T)

# 3) Matrice de confusion
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm,
                     index=[f"true_{id2label[i]}"  for i in range(NUM_LABELS)],
                     columns=[f"pred_{id2label[i]}" for i in range(NUM_LABELS)])
print("\n=== Confusion matrix ===")
print(cm_df)

In [ ]:
# ─────────────────────────────────────────────────────────────
#  Post-Optuna : 2 seeds + SWA, test, puis full-train final
# ─────────────────────────────────────────────────────────────
import os, json, gc, math, copy, random, numpy as np, torch
from collections import OrderedDict
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model
from torch.utils.data import DataLoader

# Dossiers / fichiers Optuna déjà créés
best_params  = json.load(open(os.path.join(artifact_dir, "best_params.json")))

# ─── Helpers ─────────────────────────────────────────────────
def set_all_seeds(seed: int):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

def average_states(paths):
    avg = OrderedDict()
    for p in paths:
        sd = torch.load(p, map_location="cpu")
        for k, v in sd.items():
            avg[k] = avg.get(k, 0.) + v.float()
    for k in avg:
        avg[k] = (avg[k] / len(paths)).to(torch.float16)
    return avg

# ─── Configs réutilisables ──────────────────────────────────
use_bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if use_bf16 else torch.float16,
)
grad_acc = max(1, math.ceil(EFF_BATCH_TARGET / best_params["bsz"]))
l_cfg = LoraConfig(
    r=best_params["lora_r"],
    lora_alpha=2 * best_params["lora_r"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
)

# ─────────────────────────────────────────────────────────────
#  A.   train+val  (90 %)  →  deux seeds + SWA
# ─────────────────────────────────────────────────────────────
print("\n=========== Phase A : 2 seeds sur train+val ==========")

# mini-val interne 10 % pour l’early-stop
msg_tr, msg_va, lab_tr, lab_va = train_test_split(
    msg_tv, lab_tv, test_size=0.1, stratify=lab_tv, random_state=SEED
)
tr_enc, va_enc = enc(msg_tr), enc(msg_va)
train_ds, val_ds = TokenizedDS(tr_enc, lab_tr), TokenizedDS(va_enc, lab_va)

cls_counts = np.bincount(lab_tr, minlength=NUM_LABELS)
cls_weights = torch.tensor(
    cls_counts.sum() / (NUM_LABELS * (cls_counts + 1e-9)), dtype=torch.float
)

state_paths = []
for seed in [0, 13]:
    print(f"\n—— Re-train seed {seed} (train+val) ——")
    set_all_seeds(seed)

    base = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=NUM_LABELS,
        device_map="auto",
        quantization_config=bnb_cfg,
        attn_implementation="flash_attention_2",
    )
    base.resize_token_embeddings(VOCAB)
    base.config.pad_token_id = tok.pad_token_id
    base.config.use_cache = False
    base.config.use_paged_attention = True
    if USE_GRADIENT_CHECKPOINTING:
        base.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
    model = get_peft_model(base, copy.deepcopy(l_cfg))
    for p in model.parameters():
        if p.requires_grad:
            p.data = p.data.float()

    args = TrainingArguments(
        output_dir=os.path.join(artifact_dir, f"seed_{seed}_val"),
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=best_params["bsz"],
        per_device_eval_batch_size=32,
        gradient_accumulation_steps=grad_acc,
        warmup_ratio=0.08,
        learning_rate=best_params["lr"],
        lr_scheduler_type="cosine",
        bf16=use_bf16,
        fp16=not use_bf16,
        gradient_checkpointing=USE_GRADIENT_CHECKPOINTING,
        eval_on_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="eval_mcc",
        greater_is_better=True,
        seed=seed,
        logging_strategy="epoch",
        label_names=["labels"],
    )

    trainer = WeightedCETrainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=EARLY_STOP_PATIENCE)],
        class_weights=cls_weights,
    )
    trainer.train()

    state_path = os.path.join(artifact_dir, f"best_state_seed{seed}_val.pt")
    torch.save({k: v.cpu() for k, v in model.state_dict().items() if v.requires_grad}, state_path)
    state_paths.append(state_path)

    del trainer, model, base; gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()

In [ ]:
test_dl = DataLoader(test_ds, batch_size=32, collate_fn=collator)


In [ ]:
def eval_loader(loader, model):
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for batch in loader:
            for k in batch:
                batch[k] = batch[k].to(model.device)
            logits = model(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
            ).logits
            preds.extend(torch.argmax(logits, -1).cpu())
            true.extend(batch["labels"].cpu())
    return preds, true, cat_metrics(true, preds)

In [ ]:
import copy, torch, os

def load_lora_model(state_path: str):
    """
    Reconstruit le backbone + LoRA et recharge les poids sauvegardés.
    Rien d’autre n’est modifié.
    """
    base = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=NUM_LABELS,
        device_map="auto",
        quantization_config=bnb_cfg,
        attn_implementation="flash_attention_2",
    )
    base.resize_token_embeddings(VOCAB)
    base.config.pad_token_id = tok.pad_token_id
    base.config.use_cache = False
    base.config.use_paged_attention = True
    if USE_GRADIENT_CHECKPOINTING:
        base.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})

    model = get_peft_model(base, copy.deepcopy(l_cfg))

    state = torch.load(state_path, map_location="cpu")
    # seules les têtes LoRA ont été sauvegardées ; on les recharge sans toucher au reste
    model.load_state_dict(state, strict=False)
    model = model.to("cuda" if torch.cuda.is_available() else "cpu").eval()
    return model


# --- Évaluation -------------------------------------------------------------

results = {}

for state_path in state_paths:          # state_paths = ["...seed0_val.pt", "...seed13_val.pt"]
    model = load_lora_model(state_path)
    _, _, metrics = eval_loader(test_dl, model)   # ou val_loader/test_loader, selon ton besoin
    results[os.path.basename(state_path)] = metrics

# Petit récapitulatif lisible :
for name, m in results.items():
    print(f"{name:25s} | MCC={m['mcc']:.4f} | Acc={m['accuracy']:.4f} | F1={m['f1_macro']:.4f} | F1={m['f1_micro']:.4f}")

In [ ]:

# — SWA hors-ligne —
swa_state = average_states(state_paths)
torch.save(swa_state, os.path.join(artifact_dir, "best_state_swa_val.pt"))

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

if tokenizer.pad_token is None:               # c’est le cas le plus fréquent
    tokenizer.pad_token = tokenizer.eos_token

# — évaluation sur test —
print("\n=========== Phase B : évaluation SWA sur test =========")
base = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    device_map="auto",
    quantization_config=bnb_cfg,
    attn_implementation="flash_attention_2",
)
base.resize_token_embeddings(VOCAB)



base.config.pad_token_id = tokenizer.pad_token_id

model_swa = get_peft_model(base, copy.deepcopy(l_cfg))
model_swa.load_state_dict(torch.load(os.path.join(artifact_dir, "best_state_swa_val.pt"), map_location="cpu"), strict=False)



test_ds = TokenizedDS(te_enc, lab_te)
test_dl = DataLoader(test_ds, batch_size=32, collate_fn=collator)

preds, true, test_metrics = eval_loader(test_dl, model_swa)
print("\n=== Résultats test (SWA train+val) ===")
for k, v in test_metrics.items():
    print(f"{k}: {v:.4f}")

# ─────────────────────────────────────────────────────────────
#  C.   FULL TRAIN (100 %)  →  deux seeds + SWA  →  production
# ─────────────────────────────────────────────────────────────
print("\n=========== Phase C : full-train 100 % pour la prod =========")

full_enc = enc(msg_tv + msg_te)
full_ds  = TokenizedDS(full_enc, lab_tv + lab_te)

cls_counts_all = np.bincount(lab_tv + lab_te, minlength=NUM_LABELS)
cls_weights_all = torch.tensor(
    cls_counts_all.sum() / (NUM_LABELS * (cls_counts_all + 1e-9)), dtype=torch.float
)

state_paths_full = []
for seed in [0, 13]:
    print(f"\n—— Full-train seed {seed} ——")
    set_all_seeds(seed)

    base = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=NUM_LABELS,
        device_map="auto",
        quantization_config=bnb_cfg,
        attn_implementation="flash_attention_2",
    )
    base.resize_token_embeddings(VOCAB)
    base.config.pad_token_id = tok.pad_token_id
    base.config.use_cache = False
    base.config.use_paged_attention = True
    if USE_GRADIENT_CHECKPOINTING:
        base.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
    model = get_peft_model(base, copy.deepcopy(l_cfg))
    for p in model.parameters():
        if p.requires_grad:
            p.data = p.data.float()

    # pas de validation → epochs fixes
    args_prod = TrainingArguments(
        output_dir=os.path.join(artifact_dir, f"seed_{seed}_full"),
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=best_params["bsz"],
        gradient_accumulation_steps=grad_acc,
        warmup_ratio=0.08,
        learning_rate=best_params["lr"],
        lr_scheduler_type="cosine",
        bf16=use_bf16,
        fp16=not use_bf16,
        gradient_checkpointing=USE_GRADIENT_CHECKPOINTING,
        eval_strategy="no",
        save_strategy="epoch",
        save_total_limit=1,
        logging_strategy="epoch",
        seed=seed,
        label_names=["labels"],
    )

    trainer_prod = WeightedCETrainer(
        model=model,
        args=args_prod,
        train_dataset=full_ds,
        data_collator=collator,
        class_weights=cls_weights_all,
    )
    trainer_prod.train()

    state_path = os.path.join(artifact_dir, f"best_state_seed{seed}_full.pt")
    torch.save({k: v.cpu() for k, v in model.state_dict().items() if v.requires_grad}, state_path)
    state_paths_full.append(state_path)

    del trainer_prod, model, base; gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()

swa_state_full = average_states(state_paths_full)
torch.save(swa_state_full, os.path.join(artifact_dir, "best_state_swa_full.pt"))
print("\n✓ Modèle SWA full-train enregistré : best_state_swa_full.pt")

# Optionnel : sauvegarde du tokenizer / meta-infos
tok.save_pretrained(os.path.join(artifact_dir, "tokenizer_final"))
json.dump(
    {
        "best_params": best_params,
        "test_metrics": test_metrics,
        "notes": "adapter SWA train+val, puis SWA full-train pour production",
    },
    open(os.path.join(artifact_dir, "run_summary.json"), "w"),
    indent=2,
)
print("Résumé sauvegardé.")

In [ ]:
# helpers_swa.py --------------------------------------------------------------
import os, copy, torch
from peft import get_peft_model, LoraConfig
from transformers import AutoModelForSequenceClassification, BitsAndBytesConfig

def load_base(model_name, num_labels, tok, bnb_cfg, lora_cfg):
    base = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
        device_map="auto",
        quantization_config=bnb_cfg,
        attn_implementation="flash_attention_2",
    )
    base.resize_token_embeddings(tok.vocab_size)
    base.config.pad_token_id = tok.pad_token_id
    base.config.use_cache = False
    base.config.use_paged_attention = True
    return get_peft_model(base, copy.deepcopy(lora_cfg))

def average_states(state_paths):
    from collections import OrderedDict
    avg = OrderedDict()
    for p in state_paths:
        sd = torch.load(p, map_location="cpu")
        for k, v in sd.items():
            avg[k] = avg.get(k, 0.) + v.float()
    for k in avg:
        avg[k] = (avg[k] / len(state_paths)).to(torch.float16)
    return avg

In [ ]:
# run_swa_eval.py -------------------------------------------------------------
import os, torch, copy
from torch.utils.data import DataLoader
from peft import LoraConfig
# -----------------------------------------------------------------------------
artifact_dir = "./outputs"
state_paths = [
    os.path.join(artifact_dir, "best_state_seed0_val.pt"),
    os.path.join(artifact_dir, "best_state_seed13_val.pt"),
]

# ─ config réutilisée (exactement celle de ton script initial) ─
l_cfg        = LoraConfig(           # mêmes valeurs qu’avant
    r=8, lora_alpha=16, lora_dropout=0.05,
    bias="none", task_type="SEQ_CLS",
    target_modules=["q_proj","k_proj","v_proj","o_proj",
                    "gate_proj","up_proj","down_proj"],
)
use_bf16     = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
bnb_cfg      = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if use_bf16 else torch.float16,
)
# -----------------------------------------------------------------------------

# 1. SWA hors-ligne
swa_state = average_states(state_paths)
swa_path  = os.path.join(artifact_dir, "best_state_swa_val.pt")
torch.save(swa_state, swa_path)
print(f"✓ SWA sauvegardé : {swa_path}")

# 2. Évaluation
model = load_base(MODEL_NAME, NUM_LABELS, tok, bnb_cfg, l_cfg)
model.load_state_dict(torch.load(swa_path, map_location="cpu"), strict=False)

test_ds  = TokenizedDS(te_enc, lab_te)
test_dl  = DataLoader(test_ds, batch_size=32, collate_fn=collator)

preds, true, metrics = eval_loader(test_dl)
print("\n=== Résultats test (SWA train+val) ===")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

In [ ]:
!pip install langdetect

In [ ]:
#!/usr/bin/env python3
"""
Process Facebook & Instagram CSV exports (v2)

Différences principales :
1. Chemin du modèle → dossier `outputs/final_model/` issu de trainer.save_model().
2. Prompt identique à l’entraînement : add_generation_prompt=False.
3. BitsAndBytes : dtype = bfloat16 si dispo, sinon float16.
4. Tout le reste (reprise, sauvegarde incrémentale, détection de langue)
   reste inchangé.
"""
from __future__ import annotations

# ──────────────────────────────────────────────────────────────────────────
# Early env tweaks
# ──────────────────────────────────────────────────────────────────────────
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"        # supprime l’avertissement

# ──────────────────────────────────────────────────────────────────────────
# Standard lib
# ──────────────────────────────────────────────────────────────────────────
import json
import logging
import math
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
from typing import List, Sequence

# ──────────────────────────────────────────────────────────────────────────
# Third-party
# ──────────────────────────────────────────────────────────────────────────
import warnings
import pandas as pd
from langdetect import detect, LangDetectException
from tqdm.auto import tqdm
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoConfig,
    BitsAndBytesConfig,
)
from peft import PeftModel, PeftConfig

warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

# ──────────────────────────────────────────────────────────────────────────
# USER CONFIGURATION
# ──────────────────────────────────────────────────────────────────────────
FACEBOOK_CSV   = "/home/lisst_ai/FACEBOOK_concat.csv"
INSTAGRAM_CSV  = "/home/lisst_ai/INSTAGRAM_concat.csv"

# ‼️ nouveau répertoire modèle sauvegardé par trainer.save_model()
MODEL_DIR      = "/home/lisst_ai/qlora_project/outputs/final_model"

OUT_FACEBOOK   = "/home/lisst_ai/gaza_facebook_cleaned.csv"
OUT_INSTAGRAM  = "/home/lisst_ai/gaza_instagram_cleaned.csv"

BATCH_SIZE          = 64
BATCH_SAVE_EVERY    = 100          # sauvegarde toutes les 100 batchs
NUM_WORKERS: int | None = None     # None ⇒ tous les cœurs CPU

# ──────────────────────────────────────────────────────────────────────────
# CONSTANTS & LOGGING
# ──────────────────────────────────────────────────────────────────────────
SEP               = " - "
NUM_WORDS_SAMPLE  = 20
TARGET_LANG       = "en"
CONSTRUCTED_COL   = "constructed_text"
PRED_COL          = "predicted_category"
MAX_LENGTH        = 512

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s | %(levelname)-8s | %(message)s")
log = logging.getLogger("process_sm")

TRAIN_CSV = "/home/lisst_ai/gaza_classified.csv"      # fallback si id2label absent

# ──────────────────────────────────────────────────────────────────────────
# Helpers
# ──────────────────────────────────────────────────────────────────────────
def concatenate_fields(values: Sequence[str | float | None],
                       *, sep: str = SEP) -> str:
    parts: List[str] = []
    for val in values:
        if not isinstance(val, str):
            continue
        val_clean = val.strip()
        if not val_clean:
            continue
        current = sep.join(parts).lower()
        if val_clean.lower() in current:
            continue
        parts.append(val_clean)
    return sep.join(parts)

def detect_lang(sample: str) -> str | None:
    try:
        return detect(sample)
    except LangDetectException:
        return None

# — worker pour ProcessPoolExecutor (picklable) ————————————————
def _detect_lang_worker(args):
    idx, txt = args
    words = txt.split()
    sample = " ".join(words[:NUM_WORDS_SAMPLE])
    if not sample.strip():
        return idx, False
    return idx, detect_lang(sample) == TARGET_LANG
# ———————————————————————————————————————————————————————————————

def filter_english(texts: list[str],
                   *, workers: int | None = NUM_WORKERS) -> list[bool]:
    workers = workers or os.cpu_count() or 4
    log.info("Detecting language on %d texts with %d workers…",
             len(texts), workers)

    flags = [False] * len(texts)
    with ProcessPoolExecutor(max_workers=workers) as ex:
        for idx, ok in tqdm(ex.map(_detect_lang_worker,
                                   enumerate(texts),
                                   chunksize=512),
                            total=len(texts),
                            desc="Lang-detect",
                            unit="post"):
            flags[idx] = ok
    return flags

# ──────────────────────────────────────────────────────────────────────────
# Model loading & inference
# ──────────────────────────────────────────────────────────────────────────
def load_classifier(model_dir: str, *, train_csv: str = TRAIN_CSV):
    p_cfg     = PeftConfig.from_pretrained(model_dir)
    base_name = p_cfg.base_model_name_or_path

    # 1. id2label / num_labels – on tente la config d’abord
    id2label = getattr(p_cfg, "id2label", None)
    if id2label:
        num_labels = len(id2label)
    else:
        try:
            with open(Path(model_dir) / "adapter_config.json") as f:
                raw = json.load(f)
            num_labels = raw.get("num_labels")
            id2label   = {int(k): v for k, v in raw.get("id2label", {}).items()}
        except FileNotFoundError:
            num_labels = None
            id2label   = None

    # 2. Fallback : reconstruction à partir du CSV d’entraînement
    if not id2label:
        df         = pd.read_csv(train_csv, header=None, names=["text", "cat"])
        cats       = sorted(df["cat"].unique())
        id2label   = {i: c for i, c in enumerate(cats)}
        num_labels = len(id2label)
        log.info("id2label reconstruit depuis %s", train_csv)

    label2id = {v: k for k, v in id2label.items()}

    cfg = AutoConfig.from_pretrained(
        base_name,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id,
    )

    bnb_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=(
            torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
        ),
    )

    base = AutoModelForSequenceClassification.from_pretrained(
        base_name,
        config=cfg,
        device_map="auto",
        quantization_config=bnb_cfg,
        attn_implementation="flash_attention_2",
    )

    tok = AutoTokenizer.from_pretrained(base_name, padding_side="left")
    if tok.pad_token_id is None:
        tok.add_special_tokens({'pad_token': '<pad>'})
        tok.pad_token = '<pad>'
        base.resize_token_embeddings(len(tok))
    base.config.pad_token_id = tok.pad_token_id

    try:
        model = PeftModel.from_pretrained(base, model_dir)
    except RuntimeError as e:
        log.warning("Tête LoRA incompatible (%s) → ignore_mismatched_sizes=True", e)
        model = PeftModel.from_pretrained(base, model_dir,
                                          ignore_mismatched_sizes=True)
    model.eval()
    model.config.pad_token_id = tok.pad_token_id

    cats_str = ", ".join(id2label.values())
    def build_prompt(txt: str) -> str:
        messages = [
            {"role": "system",
             "content": ("You are an expert assistant. "
                         "Classify the following text into one of these "
                         f"categories: {cats_str}. "
                         "Respond with the category label only.")},
            {"role": "user", "content": txt},
        ]
        # ‼️ add_generation_prompt=False pour coller à l’entraînement
        return tok.apply_chat_template(messages,
                                       tokenize=False,
                                       add_generation_prompt=False).strip()

    return tok, model, build_prompt

# ──────────────────────────────────────────────────────────────────────────
# Batch prediction with incremental saving
# ──────────────────────────────────────────────────────────────────────────
def incremental_predict(df: pd.DataFrame,
                        tok, model, build_prompt,
                        *,
                        text_col: str = CONSTRUCTED_COL,
                        batch_size: int = BATCH_SIZE,
                        save_every: int = BATCH_SAVE_EVERY,
                        out_path: str | Path | None = None) -> None:
    """
    Remplit df[PRED_COL] in-place pour les lignes où il manque,
    en sauvegardant toutes les `save_every` batchs.
    """
    device = next(model.parameters()).device

    # indices à traiter
    to_process = df.index[df[PRED_COL].isna() | (df[PRED_COL] == "")].tolist()
    if not to_process:
        log.info("Aucune ligne à catégoriser (déjà complet).")
        return

    total_batches = math.ceil(len(to_process) / batch_size)
    batch_counter = 0

    for i in tqdm(range(0, len(to_process), batch_size),
                  desc="Batch-predict",
                  total=total_batches,
                  unit="batch",
                  ncols=300):
        batch_idx   = to_process[i: i + batch_size]
        batch_texts = df.loc[batch_idx, text_col].tolist()
        prompts     = [build_prompt(t) for t in batch_texts]

        enc = tok(prompts,
                  return_tensors="pt",
                  padding=True,
                  truncation=True,
                  max_length=MAX_LENGTH)
        enc = {k: v.to(device) for k, v in enc.items()}

        with torch.no_grad():
            logits = model(**enc).logits
        ids = torch.argmax(logits, dim=1).tolist()

        labels = [
            model.config.id2label[i]
            if isinstance(model.config.id2label, dict)
            else model.config.id2label[i]
            for i in ids
        ]
        df.loc[batch_idx, PRED_COL] = labels

        batch_counter += 1
        if out_path and batch_counter % save_every == 0:
            log.info("Interim save → %s", out_path)
            df.to_csv(out_path, index=False)

    if out_path:
        log.info("Final save → %s", out_path)
        df.to_csv(out_path, index=False)

# ──────────────────────────────────────────────────────────────────────────
# Per-platform helpers
# ──────────────────────────────────────────────────────────────────────────
def process_facebook(df: pd.DataFrame) -> pd.DataFrame:
    text_cols = ["Message", "Description", "Image Text", "Link Text"]
    df[CONSTRUCTED_COL] = df.apply(
        lambda row: concatenate_fields([row.get(c) for c in text_cols]), axis=1)
    return df

def process_instagram(df: pd.DataFrame) -> pd.DataFrame:
    text_cols = ["Description", "Image Text"]
    df[CONSTRUCTED_COL] = df.apply(
        lambda row: concatenate_fields([row.get(c) for c in text_cols]), axis=1)
    return df

# ──────────────────────────────────────────────────────────────────────────
# Robust CSV loader
# ──────────────────────────────────────────────────────────────────────────
def safe_read_csv(path: str | Path) -> pd.DataFrame:
    try:
        return pd.read_csv(path, low_memory=False)
    except pd.errors.ParserError as err:
        log.warning("Standard parser failed for %s (%s). "
                    "Retrying with engine='python'…", path, err)
        return pd.read_csv(path,
                           engine="python",
                           on_bad_lines="skip")

# ──────────────────────────────────────────────────────────────────────────
# Platform pipeline with resume capability
# ──────────────────────────────────────────────────────────────────────────
def run_pipeline(name: str,
                 src_csv: str | Path,
                 out_csv: str | Path,
                 builder_fn) -> None:
    out_csv = Path(out_csv)
    src_csv = Path(src_csv)

    if out_csv.exists():
        log.info("%s: found existing output (%s) – resume mode.", name, out_csv)
        df = safe_read_csv(out_csv)
        # reconstruit le texte si absent (rare)
        if CONSTRUCTED_COL not in df.columns:
            raw = safe_read_csv(src_csv)
            df_texts = builder_fn(raw)[[CONSTRUCTED_COL]]
            df = df.join(df_texts)
    else:
        log.info("%s: loading raw CSV…", name)
        raw = safe_read_csv(src_csv)
        df = builder_fn(raw)
        df[PRED_COL] = pd.NA                         # nouvelle colonne

    # filtrage langue pour les lignes non catégorisées
    mask_uncat = df[PRED_COL].isna() | (df[PRED_COL] == "")
    to_check   = df.loc[mask_uncat, CONSTRUCTED_COL].tolist()
    if to_check:
        log.info("%s: language filtering (%d lignes)…", name, len(to_check))
        flags = filter_english(to_check)
        df = df.loc[~mask_uncat | pd.Series(flags,
                                            index=df.loc[mask_uncat].index)
                    ].reset_index(drop=True)

    # classification
    tok, model, build_prompt = load_classifier(MODEL_DIR)
    incremental_predict(df, tok, model, build_prompt,
                        batch_size=BATCH_SIZE,
                        save_every=BATCH_SAVE_EVERY,
                        out_path=out_csv)

    log.info("%s: completed (%d rows total).", name, len(df))

# ──────────────────────────────────────────────────────────────────────────
# Main routine
# ──────────────────────────────────────────────────────────────────────────
def main() -> None:
    run_pipeline("Facebook",
                 FACEBOOK_CSV,
                 OUT_FACEBOOK,
                 process_facebook)

    run_pipeline("Instagram",
                 INSTAGRAM_CSV,
                 OUT_INSTAGRAM,
                 process_instagram)

if __name__ == "__main__":
    main()

In [ ]:
sklearn.model_selection.cross_val_score(..., return_estimator=True) → bootstrap des scores pour IC 95 % ; on publie toujours : « MCC 0.71 ± 0.03 ».
